In [8]:
import pandas as pd

In [9]:
file_path = "economic spread excel.xlsx"
wacc_df = pd.read_excel(file_path, sheet_name='WACC Value')
roic_df = pd.read_excel(file_path, sheet_name='ROIC Value')
spread_base = pd.read_excel(file_path, sheet_name='Economic Spread Final')

In [10]:
wacc_df.columns = wacc_df.columns.str.strip()
roic_df.columns = roic_df.columns.str.strip()

In [11]:
spread_base['Company Name'] = spread_base['Company Name'].str.strip()
wacc_pivot['Company Name'] = wacc_pivot['Company Name'].str.strip()
roic_pivot['Company Name'] = roic_pivot['Company Name'].str.strip()

NameError: name 'wacc_pivot' is not defined

In [12]:
#WACC
wacc_components = ['Re', 'Rd', 'Tax Rate', 'E/V', 'D/V']
wacc_filtered = wacc_df[wacc_df['Component'].isin(wacc_components)]

In [13]:
wacc_pivot = wacc_filtered.pivot_table(
    index='Company Name',
    columns='Component',
    values='Value',
    aggfunc='first'
).reset_index()

In [14]:
wacc_pivot['WACC'] = (
    wacc_pivot['E/V'] * wacc_pivot['Re'] +
    wacc_pivot['D/V'] * wacc_pivot['Rd'] * (1 - wacc_pivot['Tax Rate'])
)

In [15]:
#ROIC
roic_components = ['NOPAT', 'IC']
roic_filtered = roic_df[roic_df['Component'].isin(roic_components)]

In [16]:
roic_pivot = roic_filtered.pivot_table(
    index='Company Name',
    columns='Component',
    values='Value',
    aggfunc='first'
).reset_index()

In [17]:
roic_pivot['ROIC'] = roic_pivot['NOPAT'] / roic_pivot['IC']

In [18]:
spread_base.drop(columns=[col for col in ['WACC', 'ROIC', 'WACC_x', 'WACC_y', 'ROIC_x', 'ROIC_y'] if col in spread_base.columns], inplace=True)

In [19]:
# Merge WACC
spread_base = pd.merge(spread_base, wacc_pivot[['Company Name', 'WACC']], on='Company Name', how='left')

# Merge ROIC
spread_base = pd.merge(spread_base, roic_pivot[['Company Name', 'ROIC']], on='Company Name', how='left')

In [20]:
assert 'WACC' in spread_base.columns
assert 'ROIC' in spread_base.columns

spread_base['WACC'] = spread_base['WACC'] / 100

# Calculate Economic Spread
spread_base['Economic Spread'] = spread_base['ROIC'] - spread_base['WACC']

In [21]:
cols = [col for col in spread_base.columns if col != 'Economic Spread'] + ['Economic Spread']
spread_base = spread_base[cols]

In [22]:
with pd.ExcelWriter(output_path) as writer:
    wacc_df.to_excel(writer, sheet_name='WACC', index=False)
    roic_df.to_excel(writer, sheet_name='ROIC', index=False)
    spread_base.to_excel(writer, sheet_name='Economic Spread Final', index=False)

NameError: name 'output_path' is not defined

In [23]:
print(f"✅ File saved as: {output_path}")

NameError: name 'output_path' is not defined